# Series

Series is the primary building block of pandas. It is similar to NumPy but whereas in NumPy, you can only access an element by the integer value of its position in the array, pandas provides a means to access an element with your custom defined index as well. For more on the difference between NumPy and pandas series, see [this medium blog post](https://medium.com/@ericvanrees/pandas-series-objects-and-numpy-arrays-15dfe05919d7) by Eric van Rees.

Let's defined the problem we are going to solve here using pandas.

## Problem 

You are given the heights (in inch) and weights (in lbs) of 5 people as shown in the table below:

| Person | Height(inch) | Weight(lbs) |
|--------|--------------|-------------|
|    A   |     72       |      186    |
|    B   |     69       |      205    |
|    C   |     70       |      201    |
|    D   |     62       |      125    |
|    E   |     57       |      89     |

Find the Body Mass Index (BMI) for each person.